Install Ta-Lib & mpl_finance package + Mount google drive

In [ ]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
from google.colab import drive
drive.mount('/content/drive')

Import Tenserflow / Keras / Numpy / Pandas Packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

GPU Environment Setting

In [ ]:
import tensorflow as tf
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

Define some useful functions for exponential smoothing & one-hot encoding

In [ ]:
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int(classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

In [ ]:
# ################################################################################
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",# "2227", 
#             "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823",
#             "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871",
#             "5880", "6505", "9904", "9910"]

stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

Data Collection

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
#start = '2005/01/01'
start = '2012/05/02'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
# for stockno in stockNum:
#     # load 進 五大價格 一個量
#     df = pdr.DataReader(str(stockno)+'.TW', 'yahoo', start=start, end=end)
#     df.index = df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
#     # print(df.columns)
#     stock_time = str(df.index[0]).split(" ")[0]
#     #load in 融資融券
#     url = "https://api.finmindtrade.com/api/v3/data"
#     parameter = {
#         "dataset": "TaiwanStockMarginPurchaseShortSale",
#         "stock_id": str(stockno),
#         "date": stock_time,
#     }
#     data = requests.get(url, params=parameter)
#     data = data.json()
#     data = pd.DataFrame(data['data'])
#     #if 沒資料 則跳過這張股票
#     if data.empty == True:
#         print(str(stockno)+" can't find data !")
#         continue
#     ratio = []
#     b_total = []
#     s_total = []
#     for i in range(len(data)):
#         b_total.append(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i])
#         s_total.append(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])
#         ratio.append(float(data['ShortSaleLimit'][i] - data['ShortSaleTodayBalance'][i])/(data['MarginPurchaseLimit'][i] - data['MarginPurchaseTodayBalance'][i]+1))
#     data['b_total'] = b_total
#     data['s_total'] = s_total
#     data['b_s_ratio'] = ratio
#     data = data.set_index('date')
#     #合併這兩個表格
#     new_df = df.join(data)
#     new_df = new_df.drop(['stock_id'], axis=1)
#     new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
#     #處理三大法人投資情況 視情況加入
#     url = "https://api.finmindtrade.com/api/v3/data"
#     parameter = {
#         "dataset": "InstitutionalInvestorsBuySell",
#         "stock_id": str(stockno),
#         "date": stock_time,
#     }
#     data = requests.get(url, params=parameter)
#     data = data.json()
#     data = pd.DataFrame(data['data'])
#     if data.empty == True:
#         print(str(stockno)+" can't find data !")
#         continue
#     df = pd.DataFrame()
#     z = 0 
#     i = 0
#     while i < len(data)  :
#         a = data['date'][i]
#         count = 0
#         sell  = 0
#         buy = 0 
#         if i+6 < len(data)-1:
#             for j  in range(i, i+6):
#                 if data['date'][j]== a :
#                     count += 1
#                     sell += data['sell'][j]
#                     buy += data['buy'][j]
#         else:
#             for j in range(i, len(data)):
#                 if data['date'][j] == a :
#                     count += 1
#                     sell += data['sell'][j]
#                     buy += data['buy'][j]
#         df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
#         i+=count
#     # combine
#     new_df = new_df.join(df.set_index('date'))
#     new_df = new_df.dropna(axis=0,how='any')
#     new_df.reset_index(drop=True, inplace=True)
#     new_df = new_df.drop(['Note'], axis=1)
#     for columnName in ['Close','High', 'Low','Open']:
#         if str(columnName) == 'y_label' : continue
#         new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
#     pred_days = 5
#     ans = []
#     for i in range(len(new_df)-pred_days):
#         Pi = new_df['Close'][i+pred_days]
#         Pj = new_df['Close'][i]
#         if Pi > Pj : y = 1
#         else: y = 0
#         ans.append(y)
#     ans += [0]*pred_days
#     new_df['y_label']= ans
    
#     #store file
#     new_df = new_df[5:]
#     from google.colab import files
#     new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'.csv')
#     # from google.colab import files
#     # files.download('/content/drive/My_Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'.csv')
#     print(str(stockno)+" complete....")

#     def normalization(df):
#     #{'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}
#       dic = {}
#       for col in df.columns:
#           if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
#           else:
#               mean = statistics.mean(df[str(col)])
#               stdev = statistics.stdev(df[str(col)])
#               if stdev == 0: stdev = 1
#           dic[str(col)+'_mean'] = mean
#           dic[str(col)+'_stdev'] = stdev
#       return dic
#     record = pd.DataFrame()
#     days_before = 30
#     for i in range(days_before, len(new_df)):
#       record = record.append(normalization(new_df[i-days_before:i]), ignore_index=True )
#     record.to_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'mean&stdev.csv')
#     print(str(stockno)+" mean & stdev complete....")

In [ ]:
################################################################################

Load Training / Validation / Testing Data

In [ ]:
pd.options.mode.chained_assignment = None 

train_x = []
test_x = []
val_x = []
val_y = []
train_y = []
test_y = []

for stockno in stockNum:
    x = []
    if int(stockno) == 2227: continue
    df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'.csv')
    record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5(for_price)/'+str(stockno)+'mean&stdev.csv')
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    days_before = 30
    for i in range(days_before, len(df)):
        tmp = df[i-days_before:i]
        for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          mean = record[str(col)+'_mean'][i-30]
          stdev = record[str(col)+'_stdev'][i-30]
          def test(s): 
            return (float(s)-mean)/stdev
          tmp[str(col)].map(test)
        x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
    y = list(df['y_label'][days_before:]) 
    #train:val:test = 8:1:1
    tr_va = int(len(x)*8/10)
    va_te = int(len(x)*9/10)
    # print("no: ", tr_va, "no: ", va_te, "leng: ", len(y))
    train_x = train_x + x[:tr_va]
    train_y = train_y + y[:tr_va]
    val_x = val_x + x[tr_va: va_te]
    val_y = val_y + y[tr_va: va_te]
    test_x = test_x + x[va_te:]
    test_y = test_y +y[va_te:]
    print(str(stockno)+" loaded....")
    # print(len(train_x), len(val_x), len(test_x))

Convert raw data label to one-hot encoding

In [ ]:
y = [] 
for i in range(len(train_y)):
    y.append(to_onelist(str(train_y[i])))
train_y = y

y = [] 
for i in range(len(test_y)):
    y.append(to_onelist(str(test_y[i])))
test_y = y

y = [] 
for i in range(len(val_y)):
    y.append(to_onelist(str(val_y[i])))
val_y = y

In [ ]:
train_x = np.array(train_x).reshape(len(train_x), 30, 23)
test_x  = np.array(test_x).reshape(len(test_x), 30, 23)
val_x   = np.array(val_x).reshape(len(val_x), 30, 23)
train_y = np.array(train_y).reshape(len(train_y),2)
test_y  = np.array(test_y).reshape(len(test_y),2)
val_y   = np.array(val_y).reshape(len(val_y),2)

In [ ]:
def count_label_dist(arr):
    count_1 = 0 
    count_0 = 0 
    for i in range(len(arr)):
        if list(arr[i])== list([0,1]): count_1 += 1
        elif list(arr[i])== list([1,0]): count_0 += 1
    print(count_0, count_1)
    return count_0, count_1

c_0, c_1 = count_label_dist(train_y)
c_0, c_1 = count_label_dist(val_y)
c_0, c_1 = count_label_dist(test_y)

Define LSTM Model

In [ ]:
def lstm4_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

Prepare For Training

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# tf.keras.backend.clear_session()
model = lstm4_model((30, 23), 2)
callback = EarlyStopping(monitor="val_loss", patience=25, verbose=1, mode="auto")

In [ ]:
# start training
history = model.fit(train_x, train_y , epochs=150, batch_size= 256, validation_data=(val_x, val_y), callbacks=[callback, tensorboard_callback] )

In [ ]:
# save model
model.save('/content/drive/My Drive/Program_Trading_Code/lstm4_xxx.h5')

In [ ]:
# evaluate testing
prediction_y = []
truth_y = []
for i in range(len(test_y)):
  prediction_y.append(int(classes[np.argmax(model.predict(test_x[i].reshape(1,30,23)))]))
  truth_y.append(int(classes[np.argmax(test_y[i])]))
print("Accuracy: ")
print( 1 - (np.sum(np.logical_xor(np.array(prediction_y), np.array(truth_y))))/len(test_y) )

Validation Rate of Return

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871",
            "5880", "6505", "9904", "9910"]
#Set the information of data
start = '2018/08/01'
end = '2019/06/03'
#start = '2019/06/04'
#end = '2020/11/11'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
asset_at_the_start = 51 * 10**6
total_asset_present_value = 0
ROR_Validation = {}
for stock_num in stockNum:
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  Return_Test_Stock_df.reset_index()
  Return_Test_Stock_df = Return_Test_Stock_df[30:]
  # start evaluaion 
  ###############一般回測################################
  y_preds_list=[]
  y_truth_list=[]
  left_money = asset_at_the_start*(1/51)
  own_asset = left_money 
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_hold_stock = 0 
  #print(len(new_df))
  for i in range(time_slide, len(new_df)): 
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    y_truth_list.append(new_df['y_label'][i])
    y_preds_list.append(predict)
    price = int(Price_df['Close'][i]*100)/100
    if predict == 1:
      if own_stock == 0:
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
    elif predict == 0 :
      if own_stock > 0:
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
    money_record.append(left_money+own_stock*price)
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
  ROR_Validation[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
  print(stock_num)
  print(ROR_Validation[stock_num])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

In [ ]:
###
# lstm4_5_5 validation ror -> 
DIC_ROR = {'1101': -8.6096, '1102': 10.1097, '1216': -0.7935, '1301': -9.4354, '1326': -17.9471, '1402': -11.2923, '2002': -9.5477, '2105': -22.0487, '2207': 75.6757, '2301': 13.9152, '2303': -1.3974, '2308': -4.2712, '2317': 12.161, '2327': -8.3967, '2330': -7.3642, '2352': -0.7237, '2357': -24.599, '2382': 13.4822, '2395': 4.3062, '2408': -15.2948, '2412': 0.2369, '2454': -12.0878, '2474': -10.7817, '2609': -5.6983, '2610': 16.8774, '2633': 44.6408, '2801': -4.7183, '2823': -16.7678, '2880': 5.6272, '2881': -15.3308, '2882': -20.0186, '2883': -11.0375, '2884': 21.6132, '2885': -4.1252, '2886': 18.3774, '2887': -6.0796, '2888': -17.4904, '2890': 6.5775, '2891': -14.73, '2892': 2.4412, '2912': -16.2974, '3008': 0.411, '3045': 2.3242, '3711': -20.7536, '4904': 2.2686, '4938': -0.4619, '5871': -1.0963, '5880': 1.4831, '6505': -15.977, '9904': 4.9289, '9910': -14.1742}
# ROR_Validation = {'1101': -0.086096, '1102': 0.101097, '1216': -0.007935, '1301': -0.094354, '1326': -0.179471, '1402': -0.112923, '2002': -0.095477, '2105': -0.220487, '2207': 0.756757,
#  '2301': 0.139152, '2303': -0.013974, '2308': -0.042712, '2317': 0.12161, '2327': -0.08396699999999999, '2330': -0.073642, '2352': -0.007237, '2357': -0.24599000000000001, '2382': 0.134822,
#  '2395': 0.043061999999999996, '2408': -0.152948, '2412': 0.002369, '2454': -0.120878, '2474': -0.10781700000000001, '2609': -0.056983, '2610': 0.168774, '2633': 0.44640799999999997,
#  '2801': -0.047183, '2823': -0.16819299999999998, '2880': 0.056272, '2881': -0.153308, '2882': -0.200186, '2883': -0.110375, '2884': 0.216132, '2885': -0.041252000000000004, '2886': 0.18377400000000002,
#  '2887': -0.060796, '2888': -0.174904, '2890': 0.065775, '2891': -0.14730000000000001, '2892': 0.024412, '2912': -0.162974, '3008': 0.00411, '3045': 0.023242, '3711': -0.207536,
#  '4904': 0.022686, '4938': -0.004619, '5871': -0.010963, '5880': 0.014831, '6505': -0.15977, '9904': 0.049289, '9910': -0.141742}

In [ ]:
# validation ROR to csv
DIC_ROR = {'1101': -8.6096, '1102': 10.1097, '1216': -0.7935, '1301': -9.4354, '1326': -17.9471, '1402': -11.2923, '2002': -9.5477, '2105': -22.0487, '2207': 75.6757, '2301': 13.9152, '2303': -1.3974, '2308': -4.2712, '2317': 12.161, '2327': -8.3967, '2330': -7.3642, '2352': -0.7237, '2357': -24.599, '2382': 13.4822, '2395': 4.3062, '2408': -15.2948, '2412': 0.2369, '2454': -12.0878, '2474': -10.7817, '2609': -5.6983, '2610': 16.8774, '2633': 44.6408, '2801': -4.7183, '2823': -16.7678, '2880': 5.6272, '2881': -15.3308, '2882': -20.0186, '2883': -11.0375, '2884': 21.6132, '2885': -4.1252, '2886': 18.3774, '2887': -6.0796, '2888': -17.4904, '2890': 6.5775, '2891': -14.73, '2892': 2.4412, '2912': -16.2974, '3008': 0.411, '3045': 2.3242, '3711': -20.7536, '4904': 2.2686, '4938': -0.4619, '5871': -1.0963, '5880': 1.4831, '6505': -15.977, '9904': 4.9289, '9910': -14.1742}
Validation_ROR_df = pd.DataFrame(columns =['Name', 'weights'])
for key in DIC_ROR:
  Validation_ROR_df = Validation_ROR_df.append({'Name': key, 'weights': DIC_ROR[key]}, ignore_index=True)
Validation_ROR_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Validation_ROR_df.csv')

Convert Rate of Return to Weights

In [ ]:
Weights_Dict = {}
for key in DIC_ROR:
  # print(key, '->', DIC_ROR[key])
  if DIC_ROR[key] > 0:
    Weights_Dict[key] = DIC_ROR[key]
WeightSum = sum(Weights_Dict.values())
NewWeightSum = 0
for key in Weights_Dict:
  Weights_Dict[key] /= WeightSum
  print(key, '->', Weights_Dict[key])
  NewWeightSum += Weights_Dict[key]
print(Weights_Dict)
print(NewWeightSum)

In [ ]:
###
# lstm 4_5_5 Weights for testing ->
Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}
0.9999999999999999

Save the weights for testing to csv file

In [ ]:
Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",# "2227", 
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871",
            "5880", "6505", "9904", "9910"]
Return_Dict = {}
for key in stockNum:
  if key not in Weights_Dict.keys(): Return_Dict[key] = 0.0
  else: Return_Dict[key] = Weights_Dict[key]
Return_Weight_df = pd.DataFrame(columns =['Name', 'weights'])
for key in Return_Dict:
  Return_Weight_df = Return_Weight_df.append({'Name': key, 'weights': Return_Dict[key]}, ignore_index=True)
Return_Weight_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Weight_df.csv')

Find the Rate of Return in Testing

In [ ]:
#選定欲回測的model
from keras.models import load_model
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
model = load_model('/content/drive/My Drive/Program_Trading_Code/lstm4_5_5.h5')
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207", #"2227", 
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871",
            "5880", "6505", "9904", "9910"]
start = '2019/06/25'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

In [ ]:
Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}

In [ ]:
asset_at_the_start = 19 * 10**6
total_asset_present_value = 0
ROR_Test = {}

time_point_asset = [0]*276
for stock_num in stockNum:
  if stock_num not in Weights_Dict.keys(): continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 10
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  Return_Test_Stock_df.reset_index()
  Return_Test_Stock_df = Return_Test_Stock_df[30:]
  # start evaluaion 
  ###############一般回測################################
  y_preds_list=[]
  y_truth_list=[]
  if stock_num not in Weights_Dict.keys(): continue
  left_money = asset_at_the_start * Weights_Dict[stock_num] #10**6
  own_asset = left_money #10**6
  time_slide = 30 
  own_stock = 0 
  money_record = [left_money]*time_slide
  origin_hold_stock = 0 
  print(len(new_df))
  for i in range(time_slide, len(new_df)): 
    # print("長度", len(new_df)-time_slide)
    tmp = new_df[i-time_slide:i]
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model.predict(x.reshape(1,30,23)))])
    y_truth_list.append(new_df['y_label'][i])
    y_preds_list.append(predict)
    price = int(Price_df['Close'][i]*100)/100
    if predict == 1:
      if own_stock == 0:
        own_stock += int( left_money / ( price * (1 + 0.001425)) )
        left_money = left_money - own_stock*price*(1 + 0.001425)
    elif predict == 0 :
      if own_stock > 0:
        left_money += own_stock*price*(1 - 0.001425 - 0.003)
        own_stock = 0
    money_record.append(left_money+own_stock*price)
    time_point_asset[i-30]+=left_money+own_stock*price
  print(str(stock_num)+"最後資產 ：", int(money_record[-1]),"報酬率：",(int(money_record[-1])-(own_asset))*100/(own_asset),"%")
  total_asset_present_value += int(money_record[-1])
  # insert y label and y truth
  #Return_Test_Stock_df.insert(6, 'y_pred', y_preds_list)
  #Return_Test_Stock_df.insert(7, 'y_true', y_truth_list)
  #Return_Test_Stock_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Test_Stock_DF/Return_Test_Stock_df'+str(stock_num)+'.csv')
  ROR_Test[stock_num] = ((int(money_record[-1])-(own_asset))*100/(own_asset))/100
  print(stock_num)
  print(ROR_Test[stock_num])
print("測試結束")
print("our總投資報酬率：", (total_asset_present_value - asset_at_the_start)*100 / asset_at_the_start, "%")

In [ ]:
total_df = pd.DataFrame(columns =['date','profit_rate'])

df  = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/Return_Test_Stock_DF/Return_Test_Stock_df'+str(2395)+'.csv')
date = list(df['Date'])
# time_point_asset = [19*1000000]*30 + time_point_asset

print(len(time_point_asset), len(date))

for i in range(len(date)):
  print(time_point_asset[i]/19*1000000)
  total_df = total_df.append({'date': date[i], 'profit_rate': (time_point_asset[i]-19*1000000)*100/(19*1000000) }, ignore_index=True)
total_df.to_csv('/content/drive/My Drive/Program_Trading_Code/total19.csv')

In [ ]:
Weights_Dict = {'1102': 0.0392674671615576, '2207': 0.2939348412591753, '2301': 0.054048553275221456, '2317': 0.0472349988774842, '2382': 0.05236672164016261, '2395': 0.016725873872726126, '2412': 0.0009201522271257303, '2610': 0.06555414604513213, '2633': 0.17339101536797932, '2880': 0.021856819807859477, '2884': 0.08394864548465104, '2886': 0.0713803526330958, '2890': 0.02554791588822073, '2892': 0.009481957015024621, '3008': 0.0015963806051020478, '3045': 0.009027512901163454, '4904': 0.008811554843636268, '5880': 0.005760564660405955, '9904': 0.019144526434276114}
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",# "2227", 
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382", "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2633", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888", "2890", "2891", "2892", "2912", "3008", "3045", "3711", "4904", "4938", "5871",
            "5880", "6505", "9904", "9910"]
Return_Dict = {}
for key in stockNum:
  if key not in Weights_Dict.keys(): Return_Dict[key] = 0.0
  else: Return_Dict[key] = Weights_Dict[key]
Return_Weight_df = pd.DataFrame(columns =['Name', 'weights', 'Test_ROR'])
for key in Return_Dict:
  if key not in ROR_Test.keys(): Return_Weight_df = Return_Weight_df.append({'Name': key, 'weights': Return_Dict[key], 'Test_ROR': 0.0}, ignore_index=True)
  else: Return_Weight_df = Return_Weight_df.append({'Name': key, 'weights': Return_Dict[key], 'Test_ROR': ROR_Test[key]}, ignore_index=True)
Return_Weight_df.to_csv('/content/drive/My Drive/Program_Trading_Code/Return_Weight_df.csv')

In [ ]:
predict = 5
predict = 1 if predict >= 3 else 0
print(predict)

addition_money = 5
addition_money = 0 if addition_money < 0 else addition_money
print(addition_money)